In [1]:
import logging
import pandas as pd
import numpy as np
import configparser

from simple_logging.custom_logging import setup_custom_logger
from input_reader.input_reader import get_input

settingsFile = "settings.cfg"

# logger einrichten
LOGGING_LEVEL = logging.INFO
logger = setup_custom_logger('GM_LOGGER', LOGGING_LEVEL, flog="logs/gm.log")

In [2]:
pd.__version__

'0.18.1'

In [3]:
# get the input
(stores_pd, stores_migros_pd, drivetimes_pd, drivetimes_migros_pd, haushalt_pd) = get_input(settingsFile, logger)

In [4]:
drivetimes_pd.head(3)

,fahrzeit,hektar_id
filiale_id,,
SM_MIG_61607_15939,21,61341718
SM_MIG_61607_15939,21,61341719
SM_MIG_61607_15939,21,61341722


In [5]:
print('Number of entries in drivetimes %d: ' % len(drivetimes_pd))

Number of entries in drivetimes 110299436: 


# Duplikate in drivetimes

In [7]:
drivetimes_pd2 = drivetimes_pd.reset_index()
drivetimes_pd2 = drivetimes_pd2.set_index(['filiale_id', 'fahrzeit', 'hektar_id'])

In [8]:
drivetimes_pd2.head(3)

Empty DataFrame
Columns: []
Index: [(SM_MIG_61607_15939, 21, 61341718), (SM_MIG_61607_15939, 21, 61341719), (SM_MIG_61607_15939, 21, 61341722)]

In [50]:
drivetimes_duplicates = drivetimes_pd2[ drivetimes_pd2.index.duplicated()]

In [51]:
print('Number of duplicates: %d' % len(drivetimes_duplicates))

Number of duplicates: 369841


In [70]:
drivetimes_duplicates.head(10)

Empty DataFrame
Columns: []
Index: [(SM_DEN_76515_25781, 26, 73722539), (SM_DEN_76515_25781, 23, 74552662), (SM_DEN_76515_25781, 21, 75792395), (SM_DEN_76515_25781, 10, 75972610), (SM_DEN_76515_25781, 24, 74552664), (SM_DEN_76515_25781, 30, 74302685), (SM_DEN_76515_25781, 30, 75142489), (SM_DEN_76515_25781, 24, 74552666), (SM_DEN_76515_25781, 11, 75972613), (SM_DEN_76515_25781, 7, 76332563)]

In [76]:
a=drivetimes_pd.loc['SM_MIG_68294_26394']
a[ (a['fahrzeit']==11) & (a['hektar_id']==67672614) ]

,fahrzeit,hektar_id
filiale_id,,
SM_MIG_68294_26394,11,67672614
SM_MIG_68294_26394,11,67672614


In [71]:
drivetimes_duplicates.to_csv('/userdata/pmavrodi/Projekte/Stao/output/drivetimes_duplicates.csv')

# Duplikate in drivetimes_migros_pd

In [78]:
drivetimes_pd3 = drivetimes_migros_pd.reset_index()
drivetimes_pd3 = drivetimes_pd3.set_index(['filiale_id', 'fahrzeit', 'hektar_id'])

In [79]:
drivetimes_pd3.head(4)

Empty DataFrame
Columns: []
Index: [(SM_MIG_61607_15939, 21, 61341718), (SM_MIG_61607_15939, 21, 61341719), (SM_MIG_61607_15939, 21, 61341722), (SM_MIG_61607_15939, 22, 61341730)]

In [80]:
drivetimes_duplicates2 = drivetimes_pd3[ drivetimes_pd3.index.duplicated()]

In [81]:
print('Number of duplicates: %d' % len(drivetimes_duplicates2))

Number of duplicates: 32660


In [82]:
drivetimes_duplicates2.head(3)

Empty DataFrame
Columns: []
Index: [(SM_MIG_68294_26394, 27, 69992480), (SM_MIG_68294_26394, 27, 69992482), (SM_MIG_68294_26394, 26, 69992486)]

# Filiale in drivetimes, aber nicht in stores_sm. Betreffen sind LAT, RLAT, usw

In [17]:
# get all relevant hektars, i.e. those from which a Migros store is reachable
relevant_hektars = set(drivetimes_migros_pd['hektar_id'])

In [19]:
drivetimes_rel_hektars_pd = drivetimes_pd[drivetimes_pd['hektar_id'].isin(relevant_hektars)]

In [84]:
drivetimes_rel_hektars_stores_pd = drivetimes_rel_hektars_pd.join(
        stores_pd[['FORMAT', 'VERKAUFSFLAECHE', 'VERKAUFSFLAECHE_TOTAL', 'RELEVANZ']], how='left')
len(drivetimes_rel_hektars_pd)

110246169

In [47]:
b=drivetimes_rel_hektars_stores_pd[pd.isnull(drivetimes_rel_hektars_stores_pd['VERKAUFSFLAECHE_TOTAL']) &
                                            pd.isnull(drivetimes_rel_hektars_stores_pd['RELEVANZ'])&
                                            pd.isnull(drivetimes_rel_hektars_stores_pd['FORMAT'])].index.unique()

In [48]:
b

array(['SM_MIG_49634_10888', 'SM_MIG_49929_11561', 'SM_MIG_50409_11684',
       'SM_MIG_52163_14753', 'SM_MIG_53455_15390', 'SM_MIG_59382_24561',
       'SM_MIG_60054_19967', 'SM_MIG_60154_20159', 'SM_MIG_60395_20720',
       'SM_MIG_61188_25978', 'SM_MIG_61241_26553', 'SM_MIG_61415_21212',
       'SM_MIG_62157_26119', 'SM_MIG_62646_22987', 'SM_MIG_63564_24405',
       'SM_MIG_63698_23985', 'SM_MIG_65022_22482', 'SM_MIG_66122_21627',
       'SM_MIG_66884_24500', 'SM_MIG_69125_26380', 'SM_MIG_69338_24842',
       'SM_MIG_69968_26167', 'SM_MIG_69969_26160', 'SM_MIG_70294_11367',
       'SM_MIG_70708_23453', 'SM_MIG_71544_10197', 'SM_MIG_71749_11257',
       'SM_MIG_72126_25738', 'SM_MIG_73994_26798', 'SM_MIG_74138_25325',
       'SM_MIG_75426_22570', 'SM_MIG_75801_19047'], dtype=object)

In [55]:
drivetimes_rel_hektars_stores_pd[~pd.isnull(drivetimes_rel_hektars_stores_pd['VERKAUFSFLAECHE_TOTAL'])].head(2)

,fahrzeit,hektar_id,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ
SM_ALD_49736_11976,7,49971200,Aldi Supermarkt,115.0,1000.0,0.8
SM_ALD_49736_11976,7,49971201,Aldi Supermarkt,115.0,1000.0,0.8


In [91]:
np.savetxt('/userdata/pmavrodi/Projekte/Stao/output/filiale_nur_drivetimes.txt', b,  fmt='%s')

# Hektaren von drivetimes_sm ohne HH. Betrifft ist (lokal) Umsatzpotenztal

In [58]:
enriched_pd = drivetimes_rel_hektars_stores_pd[~pd.isnull(drivetimes_rel_hektars_stores_pd['VERKAUFSFLAECHE_TOTAL'])].join(haushalt_pd['H14PTOT'], on='hektar_id')

In [60]:
enriched_pd.head(5)

,fahrzeit,hektar_id,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ,H14PTOT
SM_ALD_49736_11976,7,49971200,Aldi Supermarkt,115.0,1000.0,0.8,NaN
SM_ALD_49736_11976,7,49971201,Aldi Supermarkt,115.0,1000.0,0.8,NaN
SM_ALD_49736_11976,7,49971204,Aldi Supermarkt,115.0,1000.0,0.8,NaN
SM_ALD_49736_11976,28,50921477,Aldi Supermarkt,115.0,1000.0,0.8,1.0
SM_ALD_49736_11976,8,49971206,Aldi Supermarkt,115.0,1000.0,0.8,2.0


In [64]:
hektars_ohne_HH_info = enriched_pd[pd.isnull(enriched_pd['H14PTOT'])]['hektar_id'].unique()

In [97]:
len(hektars_ohne_HH_info)

54459

In [94]:
np.savetxt('/userdata/pmavrodi/Projekte/Stao/output/hektaren_ohne_HH_info.txt', hektars_ohne_HH_info, fmt='%s')